In [1]:
pip install opencv-python facenet-pytorch numpy


In [10]:
import os
import cv2
import torch
import numpy as np
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image

# Load MTCNN for face detection and InceptionResnetV1 for face recognition
mtcnn = MTCNN(keep_all=True)
model = InceptionResnetV1(pretrained='vggface2').eval()

# Function to extract face embeddings
def extract_face_embeddings(image):
    img_cropped = mtcnn(image)
    if img_cropped is not None:
        img_embedding = model(img_cropped.unsqueeze(0))
        return img_embedding
    return None

# Function to recognize faces
def recognize_face(image, known_faces, known_names, threshold=0.6):
    img_embedding = extract_face_embeddings(image)
    if img_embedding is None:
        return "No face detected."

    min_dist = float('inf')
    recognized_name = "Unknown"

    for i, known_embedding in enumerate(known_faces):
        dist = torch.dist(img_embedding, known_embedding)
        if dist < min_dist:
            min_dist = dist
            recognized_name = known_names[i]

    if min_dist > threshold:
        recognized_name = "Unknown"

    return recognized_name

# Load known faces and their names
def load_known_faces(known_faces_dir):
    known_faces = []
    known_names = []
    for filename in os.listdir(known_faces_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            name = os.path.splitext(filename)[0]
            image = Image.open(os.path.join(known_faces_dir, filename))
            known_faces.append(extract_face_embeddings(image))
            known_names.append(name)
    return known_faces, known_names

# Update this path to your known faces directory
known_faces_dir = '/known_faces.zip'  # Example: 'known_faces'
known_faces, known_names = load_known_faces(known_faces_dir)

# Real-time face detection and recognition
cap = cv2.VideoCapture(0)  # Use 0 for webcam or provide the path to a video file

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to PIL format
    frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Detect faces
    boxes, _ = mtcnn.detect(frame_pil)

    if boxes is not None:
        for box in boxes:
            x1, y1, x2, y2 = [int(b) for b in box]
            face_img = frame_pil.crop((x1, y1, x2, y2))
            name = recognize_face(face_img, known_faces, known_names)

            # Draw rectangle around the face
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            # Put name label
            cv2.putText(frame, name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Display the resulting frame
    cv2.imshow('Face Detection and Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
cap.release()
cv2.destroyAllWindows()


NotADirectoryError: [Errno 20] Not a directory: '/known_faces.zip'